# HW1 Multi Location Vacuum Problem (60%)
Like example 2-1, but we want to add mobility of the vacuum robot to clean the place. By expanding to two dimensional space and up to 9-places, the problem may be more complex.
<img src="images/hw2-1_multi_location_vacuum_problem.png">

In this question, you need to define the problem by yourself. The problem is limited by following rule:
- The vacuum robot agent can go **(Left, Right, Up, Down)** and clean the place by **Sucking** action.
- The place can be index 0~8 from left-top to right-bottom.
- The goal is that: there is no dirt in all place.
- If robot is located on edge of places, the going outside action **is not allowed**.
- If robot is located on the clean place, the clean action **is not allowed**.


## 1. Import python package

In [1]:
import math
import sys

# For some data structure implementation
import heapq
from collections import defaultdict, deque, Counter

## 2. Problem class definition

In [2]:
class Problem(object):
    def __init__(self, initial=None, goal=None, **other_keywords):
        """Specify the initial and goal states.
        Subclasses can use other keywords if they want."""
        self.__dict__.update(initial=initial, goal=goal, **other_keywords) 

    def actions(self, state):           raise NotImplementedError
    def result(self, state, action):    raise NotImplementedError
    def is_goal(self, state):           return state == self.goal
    def step_cost(self, s, action, s1): return 1
    def h(self, node):                  return 0

## 3. Node definition

In [3]:
class Node:
    '''A Node in a search tree.'''
    def __init__(self, state, parent=None, action=None, path_cost=0):
        # __dict__ store this object's all attributes
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
    
    '''All Reserve words are not introduced here. If you are interest in them, please Google them'''
    # __repr__ is a built-in function used to compute the '''official''' string reputation of an object.
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.state < other.state
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deeepening search was cut off.

def expand(problem, node):
    '''Expand a node, generating the children nodes.'''
    s = node.state
    for action in problem.actions(s): 
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.step_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    '''The sequence of actions to get to this node.'''
    if node.parent is None:
        return []
    else: 
        return path_actions(node.parent) + [node.action]


def path_states(node):
    '''The sequence of states to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path_states(node.parent)) + [node.state]


def path(node):
    '''Alternating states and actions to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path(node.parent) + [node.action] ) + [node.state]

## 4. Search Algorithms

In [4]:
FIFOQueue = deque
LIFOQueue = list

def depth_limited_search(problem, limit=9):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    solution = failure
    while frontier:
        node = frontier.pop()
        if len(node) > limit:
            solution = cutoff
        else:
            for child in expand(problem, node):
                if problem.is_goal(child.state):
                    return child
                frontier.append(child)
    return solution

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    frontier = FIFOQueue([Node(problem.initial)])
    reached = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure
        

In [5]:
dirt  = '*'
clean = ' '

class MultiLocationVacuumProblem(Problem):    
    def actions(self, state): 
        '''
        
        return ???
        '''
        (loc, *dirt) = state
        loc = int(loc)
        
        action_set = ['L', 'R', 'U', 'D', 'S']
        if loc in (0, 3, 6): action_set.remove('L')
        if loc in (0, 1, 2): action_set.remove('U')
        if loc in (6, 7, 8): action_set.remove('D')
        if loc in (2, 5, 8): action_set.remove('R')
        if dirt[loc] == clean: action_set.remove('S')
        return (tuple(action_set))
    
    def is_goal(self, state):
        return dirt not in state
    
    def result(self, state, action):   
        '''
        (loc, dirtL, dirtR) = state
        if   action == ???: return ???
        elif action == ???: return ???
        
          
        else: raise ValueRrror('unknown action: ' + action)
        '''
        (loc, *dirt) = state
        loc = int(loc)
        if action == 'L':
            if loc not in (0, 3, 6): loc = loc - 1
            return (str(loc), *dirt)
        elif action == 'R':
            if loc not in (2, 5, 8): loc = loc + 1
            return (str(loc), *dirt)
        elif action == 'U':
            if loc not in (0, 1, 2): loc = loc - 3
            return (str(loc), *dirt)
        elif action == 'D':
            if loc not in (6, 7, 8): loc = loc + 3
            return (str(loc), *dirt)
        elif action == 'S':
            dirt[loc] = clean
            return (str(loc), *dirt)
        
        else: raise ValueError('unknown action: ' + action)
        

In [6]:
initial_state = (0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')


p1 = MultiLocationVacuumProblem(initial_state)
print(p1.initial)
p1.result(p1.initial, 'D')

(0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')


('3', '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')

In [7]:
initial_state = (0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')
p1 = MultiLocationVacuumProblem(initial=initial_state)

# Apply the DFS on the vacuum problem
# result_graph = depth_limited_search(problem=p1, limit=20)
result_graph = breadth_first_search(problem=p1)
# Take a look the state sequence of the result
path_states(result_graph)


[(0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 ('0', ' ', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 ('1', ' ', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 ('2', ' ', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 ('2', ' ', ' ', ' ', ' ', '*', ' ', '*', ' ', '*'),
 ('1', ' ', ' ', ' ', ' ', '*', ' ', '*', ' ', '*'),
 ('4', ' ', ' ', ' ', ' ', '*', ' ', '*', ' ', '*'),
 ('4', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', '*'),
 ('3', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', '*'),
 ('6', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', '*'),
 ('6', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'),
 ('7', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'),
 ('8', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'),
 ('8', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')]

In [8]:
path(result_graph)

[(0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 'S',
 ('0', ' ', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 'R',
 ('1', ' ', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 'R',
 ('2', ' ', ' ', '*', ' ', '*', ' ', '*', ' ', '*'),
 'S',
 ('2', ' ', ' ', ' ', ' ', '*', ' ', '*', ' ', '*'),
 'L',
 ('1', ' ', ' ', ' ', ' ', '*', ' ', '*', ' ', '*'),
 'D',
 ('4', ' ', ' ', ' ', ' ', '*', ' ', '*', ' ', '*'),
 'S',
 ('4', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', '*'),
 'L',
 ('3', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', '*'),
 'D',
 ('6', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', '*'),
 'S',
 ('6', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'),
 'R',
 ('7', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'),
 'R',
 ('8', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'),
 'S',
 ('8', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')]